# Einleitung
In der Bildverarbeitung stellt neben der Bildbearbeitung und der Computergrafik auch der Bereich der Computer Vision eine sehr große Rolle. Neben dem puren Erkennen von Objekten, stellt der Bereich der Tiefenwertberechnung ein weiteres komplexes Feld. Dabei werden Informationen aus einem oder mehreren Bildern dazu verwendet einen 2D-Punkt auf einem Bild einem 3D Punkt in der realen Welt zuzuordnen. Um die Auswertung zu erleichtern können auch Tiefensensoren verwendet werden. So wird zum Beispiel bei der Azure Kinect neben einer Kamera auch ein Tiefensensor verwendet um eine 3D-Konstruktion des Raumes oder die Pose eines Menschens zu erfassen.

Im Folgenden wird die Zuhilfenahme solcher Sensoren jedoch nicht berücksichtigt und sich einzig auf die Verwendung von korresponierenden Punkten in zwei Bildern beschränkt. Zudem wurden die verwendeten Kameras mittels eines Kalibrierungsmusters kalibriert und daraus deren Kameramatrizen berechnet.


In ihrem Buch _Multiple View Geometry_, beschreiben Hartley und Zisserman in Algorithmus 12.1 ein Verfahren, das mittels Triangulierung aus einem korrespondierenden 2D-Punktpaar einen möglichst genauen 3d-Punkt bestimmt. Dieser wurde im Folgenden implementiert.

# Epipolargeometrie
Die hier zu Grunde liegende Theorie stammt aus der Epipolargeometrie. So wird jeder der beiden Punktlochkameras jeder 3D-Punkt durch die Linse auf den 2D Sensor projeziert insofern dieser im Sichtfeld liegt.

![Epipolargeometrie](img/Epipolargeometrie3.png)
<br/>https://upload.wikimedia.org/wikipedia/commons/3/33/Epipolargeometrie3.svg


Ein Bildpunkt $X$ wird dabei auf die Punkte $X_R$ und $X_L$ projeziert, indem ein Strahl von X zum jeweiligen Projektionszentrum $O$ führt. Dies entspricht z.B. einem Lichstrahl, der ausgehend von $X$ durch die Linse der Kamera auf den Sensor trifft.
Die Bildebene auf der die Bildpunkte liegen sind dann die jeweigen Sensoren der Kameras.

Angenommen es wäre nur ein Bild verfügbar, so kann nicht genau bestimmt werden, wo $X$ im zweiten Bild liegen würde.

Ist nun aber die Translation und Rotation der beiden Kameras zueinander bekannt, so kann der sog. Epipol bestimmt werden.
Dieser gibt an, wo sich das jeweils andere Kamerazentrum im eigenen Bild (auf der Bildebene) befindet. Somit geben die Epipole (gelb) $e_L$ und $e_R$ die Projektion des rechten und linken Kamerazentrums auf die Bildebenen 1 und 2 an.

Daraus kann dann mittels $X_L$ bzw. $X_R$ und dem jeweiligen Epipol eine Epipolarlinie gebildet werden, auf der alle Punkte der Epipolarebene liegen.
Aus den Vektoren $\vec{Oe_L}$ und $\vec{OX_L}$ lässt sich dann mittels Kreuzprodukt eine Normale $\vec{Oe_L}\times\vec{OX_L}$ bilden, für die dann wiederum $\vec{OX_L} \cdot (\vec{Oe_L} \times \vec{OX_L}) = 0$ gelten muss (senkrecht zueinander). Dieser Zusammenhang wird als _Epipolar Constraint_ bezeichnet. Analog dazu auch bei der rechten Seite.

Diese Relation kann dann als
$$X_l^T\cdot T \cdot R \cdot X_r = 0$$
beschrieben werden, wobei $R$ die Rotation und $T$ die Rotation der linken zur rechten Kamera beschreiben und $X_l$ und $X_r$ Punkte im 3D Raum beschreiben.
Weiter kann dann $R$ und $T$ als Essential Matrix $E = T\cdot R$ zusammengefasst werden. Durch dieses Verhältnis lassen sich nun 3D-Punkte aus dem Koordinatensystem von Kamera L zu dem von Kamera R und umgekehrt durch $E^{-1}$ berechnen. Eine Umkehrung von $E$ mittels SVD ist einfach möglich, da $T$ schiefsymmetrisch und $R$ orthonormal ist.

Da $X_L$ und $X_L$ 3D-Punkte sind, die in den meisten Fällen nicht bekannt sein sollten, muss die Formel soweit umgeformt werden, sodass die 2D-Bildpunkte $x_L = [x_{L1},x_{L2}, 1]^T$ und $x_R = [x_{R1},x_{R2}, 1]^T$ transformiert werden können. Dafür wird die Projektion eines 3D-Punktes auf die 2D Bildebene benötigt. Diese Information enthält die Kameramatrix, auch Projektionsmatrix genannt. Da auch diese Parameter bei einem Bildpaar konstant sind, wird aus den beiden Kameramatrizen $K_L$, $K_R$ und der Essentiellen Matrix dann die Fundamentalmatrix $F$ gebildet. Somit gilt $E = K^T_LFK_R$. Eine Kalibrierung der Kamera und die Berechnung der Kameramatrix ist z.B. mittels eines Kalibrierungsmusters möglich. Dazu aber mehr im Kapitel Kalibrierung.

$$ x_L^T \cdot K_L^{-T} \cdot E \cdot K_R^{-1} \cdot x_R = x_L^T \cdot F \cdot x_R = 0 $$

Diese Fundementalmatrix projeziert somit einen Punkt des linken Bildes auf einen Punkt des rechten Bildes.

# Problemstellung
In der Theorie gilt die oben gezeigte Epipolar Constraint immer für korrespondierende Punkte. Dies ist in der Realität durch Distortion, ungenaue Punkte, etc. häufig allerdings nicht der Fall. Schon bei einer minimalen Abweichung laufen die beidem Vektoren $\vec{x} = \vec{OX_L}$ und $\vec{x'} = \vec{OX_R}$ aneinander vorbei und bilden keinen Schnittpunkt $X$. Damit dennoch ein möglichst guter Punkt $X$ im dreidimensionalen Raum berechnet werden kann, kommt das Prinzip der Triangulierung ins Spiel. Ein relativ einfacher Algorithmus zur Triangulierung eines Punktes $X$ würde die zwei Vektoren aufstellen und dann jeweils die Stelle bestimmen, an denen der jeweils Andere senkrecht steht. Der Mittelpunkt dieser beiden Punkte würde dann als triangulierter Schnittpunkt $X$ gelten.

[[1]](#paper) sehen das allerdings als eher schlecht an und schlagen den _Optimalen Triangulierungsalgorithmus (12.1)_ vor.
Dieser verwendet die Fundamentalmatrix um aus korrespondierenden Punktpaaren $x \leftrightarrow x'$ eine Fundamentalmatrix zu konstruieren, für die die Epipolbedingung zutrifft und somit besser geeignete homogene Punktpaare $\hat{x} \leftrightarrow \hat{x}'$ zu finden und diese dann zur Berechnung von $X$ zu verwenden.
# Experiment
## Vorbereitung
Es wurden zwei Smartphones als Kameras verwendet, wobei eine Szene von beiden Kameras gleichzeitig erfasst wurde. Die Auflösung der erlangten Fotos wurde mittels _OpenCV_ vor jeder Analyse auf $1600 \times 1200 px$ skaliert. Beide Kameras speicherten bilder im $4:3$ Format. Die Ausrichtung zeigte von oben herab auf die Szene und die Epipole lagen nicht in der sichtbaren Bildebene. 

### Fundamentalmatrix
Zur Berechnung des Algorithmus wird eine initiale Fundamentalmatrix benötigt. Diese wird durch Verwenden der _OpenCV_ Funktion `cv2.findFundamentalMat` und manuell ausgewählten Punktpaaren ($n=10$) konstruiert.

![Bilder mit Punkten](assets/with-markers.jpg)
<br/>Abb. 2: Das verwendete Bildpaar mit eingezeichneten Punkten.

Deweiteren wurde versucht analogh zu [[2]](https://docs.opencv.org/3.4/da/de9/tutorial_py_epipolar_geometry.html) mittels SIFT und Graubildern, korrespondierende Bildpunkte zu ermitteln. Aufgrund doch einiger fehlerhaften Zuordnungen wurden vorerst manuelle Punktpaare verwendet. Die in [Kalibrierung](#kalibrierung) verwendeten Ecken, wären ebenfalls mögliche Punktpaare.

Eine Berechnung aus den im Folgenden kalibrierten Kameramatrizen hätte auch verwendet werden können[[1]](#paper).

### Kallibrierung
[[1]](#paper) geben an, dass etweder die Kameramatrizen oder $F$ bekannt sein soll. Zur einfacheren Berechnung von $X$ im letzten Schritt des Algorithmus, wurden beide Kameras kalibriert. Die erforderlichen Kameramatrizen könnten auch mittels einer Konstruktion via DLT und gemessenen 3D-Punkten ermittelt werden.

Zur Kalibrierung wurden die _OpenCV_ Funktionen `cv2.findChessboardCorners` und `cv2.calibrateCamera` verwendet.
Dafür wurde ein Schachbrettmuster ausgedruckt und in die Szene gelegt. Anschließend wurde ein Bild mit beiden Kameras gemacht. Anschließend wurde das Schachbrett in eine andere Position gebracht und erneut Fotos mit beiden Kameras gemacht. Dies wurde 10 mal durchgeführt und die Kamerapositionen nicht verändert.

Anschließend wurden mittels _OpenCV_ die Ecken des Schachbretts identifiziert und an die Kalibrierungsfunktion übergeben. Diese ermittelte dann die intrinsische Kameramatrix, als auch die Rotations- und Translationsvektoren und Distortion der einzelnen Bilder. Da die verwendeten Smartphones bereits über eine einigermaßen gute Korrektur der Verzerrung haben, wurden die Bilder nicht entzerrt. Dies ist aber dennoch für den weiteren Verlauf zu empfehlen. Der Abstand der einzelnen Kacheln von $2cm$ wurde ebenfalls übergeben.

Aus den intrinsischen $3\times3$ Kameramatrizen $K$ und $K'$, und den bildspezifischen Translations ($t$)- und Rotationsvektoren($r$) wurden dann die $3 \times 4$ Projektionsmatrizen $P$ und $P'$ berechnet, wobei die Rotationsmatrix $R$ mittels Rodigues Rotationsformel aus $r$ ermittelt wurde.

$$ P = K \cdot W = K \cdot [R|t] $$

Das verwendete Bildpaar war Bildpaar $04$ (Index 3).

Die beiden Projektionsmatrizen wurden kopiert und im Triangulierungsalgorithmus als Parameter verwendet.


## Optimaler Triangulierungsalgorithmus
### Allgemeiner Ablauf
Zum Start wurden die bereits erwähnten Bilder geladen und skaliert, die Kameramatrizen festgelegt und die korrespondierenden Punkte abgelegt.

Im Anschluss wurde mittels `HZTriangulation` eine Klasse für den Algorithmus mit den o.g. Parametern initialisiert. Bei der Initialisierung wird die Fundamentalmatrix $F$ anhand der gegebenen Punkte berechnet. Das oben gezeigte Bild in Abb.2 wird hier anhand der gegebenen Bilder und Punkte mit Markern zur leichteren Auswertung gespeichert.

Für jedes 2D-Punktpaar wird der 3D-Punkt `X` berechnet und ausgegeben.

In [27]:
pip install opencv-python numpy scipy

Note: you may need to restart the kernel to use updated packages.


In [135]:
import cv2 as cv
import numpy as np
import hz_triangulation
from hz_triangulation import HZTriangulation
from src.calibrator import Calibrator
import importlib  # importlib is a module from the standard library

np.set_printoptions(precision=5, suppress=True)
importlib.reload(hz_triangulation)

frame_size = (3648, 2736)

index_first = 2
index_second = 3
img1 = cv.imread('assets/calibration_new/img-%02d.jpg' % (index_first+1))
img2 = cv.imread('assets/calibration_new/img-%02d.jpg' % (index_second+1))
img1 = cv.resize(img1, frame_size)
img2 = cv.resize(img2, frame_size)

calibrator = Calibrator('assets/calibration_new/', 'img-*.jpg',
                        frameSize=frame_size, chessboardSize=(7, 5), chessboard_square_mm=29.15)
calibrator.calibrate()

K_1, _ = calibrator.calculateProjection(index_first)
K_2, _ = calibrator.calculateProjection(index_second)

p1 = calibrator.imgpoints_as_int(index_first)
p2 = calibrator.imgpoints_as_int(index_second)
print(p1.shape, p2.shape, K_1.shape, K_2.shape)


hz = HZTriangulation(img1, img2, p1, p2, K_1, K_2)
print("F:\n{hz.F}\n".format(**locals()))
hz.save_with_markers("assets/with-markers.jpg")
points_hz = []

for i in range(0, len(p1)):
    X = hz.single_point_step(i)
    points_hz.append(X)
    print("{i:2} => x:\t{hz.x},\txp:\t{hz.xp},\tX: {X}".format(**locals()))

True
True
True
True
True
True
True
True
True
True

Camera calibrated:  0.4178394466041533
Camera Matrix:  [[2790.294373778    0.          1827.29216617 ]
 [   0.          2794.318007802 1402.015521139]
 [   0.             0.             1.         ]]
Distortion Parameters:  [[ 0.27683505  -2.987342069  0.000200696  0.000459854  6.71839633 ]]
Rotation Vectors:  (array([[-0.741706647],
       [-0.008120225],
       [-0.182648646]]), array([[-0.502829282],
       [ 0.020669438],
       [-0.270155395]]), array([[-0.761454366],
       [ 0.475764108],
       [ 1.298924242]]), array([[-0.481109249],
       [ 0.147693592],
       [ 0.626032156]]), array([[-0.205764957],
       [ 0.42302761 ],
       [ 1.380929914]]), array([[-0.38922491 ],
       [ 0.516165039],
       [ 1.495358733]]), array([[-0.689999998],
       [-0.0743859  ],
       [-0.096006657]]), array([[-0.631370067],
       [ 0.225574234],
       [ 0.904118162]]), array([[-0.316729012],
       [ 0.553199823],
       [ 1.602964386]]

### Schritte für Punktpaare (Algorithmus)
Die Funktion `hz.singlePointStep(i)` nimmt jedes einzelen Punktpaar und evaluiert hier den 3D-Punkt X, sodass die Epipolarbedingung erfüllt ist.

Hierbei wird analog zu Hartley und Zisserman folgende Schritte ausgeführt:
#### Erstellen von $T$ und $T'$ aus $x$ und $x'$
Wenn beide Bildpunkte mit den Epipolen korrespondieren, so liegen beide Punkte auf einer Linie zwischen den beiden Kamerazentren. Somit wäre es unmöglich, die genaue Position auf dieser Linie zu bestimmen. Deshalb wird hier angenommen, dass keiner der Bildpunkte auf dieser Linie liegt[[1]](#paper). Durch den Versuchsaufbau ist dies zudem nicht möglich, da sich die Kameras nicht gegenseitig sehen.

Die Autoren merken zudem an, dass wenn nur ein Punkt auf dieser Linie liegt, sich dieser Punkt im Zentrum des anderen Kamerazentrums befinden muss. Deshalb gehen sie davon aus, dass keiner der beiden Bildpunkte auf einem Epipol liegt.

Somit können die beiden Bildpunkte auf den Urspurng $(0,0,1)^T$ mittels der Translationsmatrizen für $x$ und $x'$ verschoben werden.

#### Anwendung der Translation auf $F$
Da sich durch die Translation der Bildpunkte in den Ursprung, diese auch auf die Fundamentalmatrix angewendet werden muss, wird $F$ (`hz.Fp`) neuberechnet.
$$ F' = {T'}^{-T} F T^{-1} $$

#### Berechnen der Epipole
Die Epipole $e$ und $e'$ sollen zudem auf die $x$-Achse in den Punkten $(1,0,f)^T$ und $(1,0,f')^T$ transformiert werden.

Somit werden diese mittels SVD berechnet und auf $e^2_1 + e^2_2 = 1$ normalisiert.

#### Berechnung der Rotationsmatrizen
Durch die berechneten Epipole können nun die Rotationsmatrizen $R$ und $R'$ aufgestellt werden.

Des weiternen, führte die Normalisierung dazu, dass $Re = (1,0,e_3)^T$ und $R'e' = (1,0,e'_3)^T$ gilt.
Die nun gebildeten Rotationsmatrizen rotieren somit die Epipole an die gewüschten Positionen $(1,0,f)^T$ und $(1,0,f')^T$ auf der $x$-Achse.

#### Anwendung der Rotation auf $F'$
Analog zur Translation muss auch die Rotation der Bildpunkte in der Fundamentalmatrix widergespiegelt werden. Dies geschieht durch:
$$ F'' = R'F'R^T $$
Die Form der Fundamentalmatrix entspricht somit der in [[1]](#paper) beschriebenen Form 12.3.

#### Setzen von Parametern
Aufgrund der vorliegenden Form von $F''$ können die folgenden Parameter abgeleitet werden:
$$ f = e3 \\ f' = e'_3 \\ a = {F''}_{22} \\ b = {F''}_{23} \\ c = {F''}_{32} \\ d = {F''}_{33}  $$

#### Berechnen von Extrema
Hartley und Zisserman verwenden nun den quadratischen Abstand (Squared Distance) um den Abstand $d^2$ einer Linie $l(t)$ durch den Punkt $(0,t,1)^T$ und den Epipol $(1,0,f)^T$ zum Ursprung in Abhängigkeit von t im linken Bild zu berechnen. Um die selbe Linie auch im rechten Bild zu erhalten wird die neue Fundamentalmatrix verwendet, sodass $l'(t) = {F''}\cdot(0,t,1)^T$. Auch hier wird eine Formel zur Berechnung des quadratischen Abstands aufgestellt.

Addiert man beide Abstandsgleichungen, so wird der Abstand minimal, wenn die Ableitung ein Minimum besitzt ($s'(t) = 0$).
Die aus der Ableitung erhaltene polynomische Formel $g(t) = 0$ wird nun verwendet um 6 Nullstellen (roots) zu finden.

#### Berechnen der Kosten und $t_{min}$
Der Realteil der berechneten Nullstellen t wird nun in die Kostenfunktion (Summe der quadratsichen Abstände $s(t)$) eingesetzt. Das $t$ mit den geringsten Kosten wird als $t_{min} gewählt.

Somit ist die Linie $l(t)$ bei $t_{min}$ am Nähesten zu den beiden Ursprüngen bzw der Punkt $(0,t_{min}, 1)$ der "optimale" zum Aufspannen von $l$ und $l'$.

#### Ermitteln von $\hat{x}$ und $\hat{x}'$
Nun werden die optimalen Punkte $\hat{x}$ und $\hat{x}'$ gesucht. Diese weisen die kürzeste Distanz zum Urspurng auf, da $x$ und $x'$ im jeweilgen Ursprung liegen und deshalb auch nach dem Abstand zu dem jeweilgen Ursprung minimiert wurde.

Dafür wird für $l(t_{min})$ und $l'(t_{min})$ ermittelt und die Nähesten Punkte $\hat{x}$ und $\hat{x}'$ zum Ursprung mittels $(a,b,c) \rightarrow (-ac,-bc, a^2+b^2)$ ermittelt.

#### Retransformation von $\hat{x}$ und $\hat{x}'$
Die zu Beginn vorgenommen Translationen und Rotationen werden nun an $\hat{x} = T^{-1}R^T\hat{x}$ und $\hat{x}' = {T'}^{-1}{R'}^T\hat{x}'$ rückgänig gemacht.

Die optimierten Koordinaten bleiben homogene Koordinaten.

#### Berechnung von $\hat{X}$
Um nun den 3D-Punkt $\hat{X}$ zu berechnen muss eine Projektion von 2D-Punkten zu 3D-Weltpunkten erfolgen.

Die von den Autoren genannte Option nennt die Direct Linear Transformation als Möglichkeit die intrinsischen und extrinsichen Eigenschaften zu schätzen.

Im Implementierungsbeispiel wurden allerdings die kalibrierten Kameramatrizen verwendet und $A$ aufzustellen und mittels SVD zu lösen.
Die resultierende homogene 3D Koordinate wurde normalisiert, sodass $\hat{X}_4 = 1$.

# Auswertung
Für die Auswerung wurden die Ergebnisse aus [Allgemeiner Ablauf](#Allgemeiner-Ablauf) verwendet.
## Tiefenwertberechnung
Bei den Tiefenwerten fällt auf, dass sich die meisten zwischen $-2000 < \hat{X}_3 < -1200$ bewegen. Da die Kameramatrix in mm kalibriert wurde, würde das einer Entfernung von $1,2m$ bis $2m$ entsprechen. Ein Ausreißer mit komplett umgekehrten Vorzeichen und viel höheren Werten wurde auch verzeichnet.

Da aber alle Punkte auf dem Tisch liegen und eine Ansicht von Oben auf die Szene gewählt wurde, scheint die nicht korrekt wiedergespiegelt worden zu sein. Relativ konstante Werte ohne große Schwankungen für $z$ wären hier zu erwarten gewesen.

Betrachtet man zusätzlich die Absolutwerte, fällt auf, dass alle Werte zu hoch liegen. So war der Abstand der Kameras zur Tischoberfläche nur ca $95cm$.

## Verschiebung in X und Y Richtung
Die Maße des Schachbrettblattes entsprechen denen eines herkömmlichen Din A4 Blattes ($297 mm x 210 mm$). Die Seiten des Schachbretts vermessen circa $233mm x 175mm$. Vergleicht man die Distanz zwischen den einzelnen Punkten ergeben sich folgende Ergebnisse:

In [139]:
def evaluate_points(points):
    dists = []
    for i, point1 in enumerate(points):
        for j, point2 in enumerate(points):
            if i == j or j<i:
                continue
            dist = hz.dist(point1,point2)
            factor = dist/calibrator.size_of_chessboard_squares_mm
            dists.append(dist)
            print("{i:2} => {j:2}: {dist:8.4f} ({factor:.2f})".format(**locals()))
    print("mean:  ", np.mean(dists))
    print("median:", np.median(dists))
    print("std:   ", np.std(dists))
    print("max:   ", np.max(dists))
    print("min:   ", np.min(dists))
evaluate_points(points_hz)

 0 =>  1:  28.9587 (0.99)
 0 =>  2:  56.9721 (1.95)
 0 =>  3:  84.4047 (2.90)
 0 =>  4: 110.8936 (3.80)
 0 =>  5: 136.9428 (4.70)
 0 =>  6: 162.6381 (5.58)
 0 =>  7:  30.2989 (1.04)
 0 =>  8:  14.4840 (0.50)
 0 =>  9:  35.9322 (1.23)
 0 => 10:  63.6383 (2.18)
 0 => 11:  90.9356 (3.12)
 0 => 12: 118.4320 (4.06)
 0 => 13: 144.9759 (4.97)
 0 => 14:  60.6173 (2.08)
 0 => 15:  35.8303 (1.23)
 0 => 16:  30.9316 (1.06)
 0 => 17:  50.0447 (1.72)
 0 => 18:  75.9264 (2.60)
 0 => 19: 102.8350 (3.53)
 0 => 20: 129.9381 (4.46)
 0 => 21:  91.3724 (3.13)
 0 => 22:  64.1134 (2.20)
 0 => 23:  47.0386 (1.61)
 0 => 24:  49.1741 (1.69)
 0 => 25:  67.5695 (2.32)
 0 => 26:  92.0552 (3.16)
 0 => 27: 118.5430 (4.07)
 0 => 28: 121.7776 (4.18)
 0 => 29:  93.1074 (3.19)
 0 => 30:  70.9785 (2.43)
 0 => 31:  62.0814 (2.13)
 0 => 32:  69.5570 (2.39)
 0 => 33:  88.3472 (3.03)
 0 => 34: 112.2455 (3.85)
 1 =>  2:  28.0135 (0.96)
 1 =>  3:  55.4461 (1.90)
 1 =>  4:  81.9351 (2.81)
 1 =>  5: 107.9844 (3.70)
 1 =>  6: 13

Hierbei fällt auf, dass vor allem Punkt $9$ stark abweicht. Jedoch stimmen die gemessenen Längenverhältnisse nicht mit den relationen des Blattes überein. Dennoch liegen die Größenordnungen schon in dem richtigen Bereich.

## Vergleich mit OpenCV


In [140]:
points_cv = hz.triangulateOpenCv().T
for i in range(0,points_cv.shape[0]):
    points_cv[i] = points_cv[i]/points_cv[i,-1]
points_cv

array([[ 680.26544,  217.88412, -526.98773,    1.     ],
       [ 684.5136 ,  220.22124, -528.4898 ,    1.     ],
       [ 688.54126,  222.68526, -530.13354,    1.     ],
       [ 692.4018 ,  225.31303, -531.9122 ,    1.     ],
       [ 696.06165,  228.07582, -533.8635 ,    1.     ],
       [ 699.5152 ,  231.02432, -535.97876,    1.     ],
       [ 702.78015,  234.18214, -538.3078 ,    1.     ],
       [ 679.6634 ,  217.13728, -522.6676 ,    1.     ],
       [ 684.0811 ,  219.35805, -524.14923,    1.     ],
       [ 688.2766 ,  221.68762, -525.7574 ,    1.     ],
       [ 692.36316,  224.19588, -527.56445,    1.     ],
       [ 696.19727,  226.81493, -529.5046 ,    1.     ],
       [ 699.85596,  229.65196, -531.67114,    1.     ],
       [ 703.2983 ,  232.63663, -533.9765 ,    1.     ],
       [ 679.08167,  216.72035, -518.22894,    1.     ],
       [ 683.6385 ,  218.81233, -519.74384,    1.     ],
       [ 688.0333 ,  221.02794, -521.2882 ,    1.     ],
       [ 692.2851 ,  223.4063 ,

Hier wurden die 3D Koordinaten mittels `cv.triangulatePoints` ermittelt.

Auch hier liegen die Absolutwerte ähnlich weit weg von der erwarteten Entfernung in $z$ Richtung und auch in die anderen beiden Richtungen.

Dies ist sogar noch extremer bemerkbar beim Betrachten der Abstände der Blattecken. Ein korrektes Verhältnis zwischen den Abständen ist auch hier nicht zu erkennen.

Auffällig ist, dass die meisten $z$-Werte hier positiv sind und sich über $x$ und $y$ mit positiven und negativen Werten mehr verteilen. 

In [141]:
evaluate_points(points_cv)

 0 =>  1:   5.0759 (0.17)
 0 =>  2:  10.0716 (0.35)
 0 =>  3:  15.0576 (0.52)
 0 =>  4:  20.0167 (0.69)
 0 =>  5:  24.9811 (0.86)
 0 =>  6:  30.0114 (1.03)
 0 =>  7:   4.4254 (0.15)
 0 =>  8:   4.9788 (0.17)
 0 =>  9:   8.9532 (0.31)
 0 => 10:  13.6574 (0.47)
 0 => 11:  18.4368 (0.63)
 0 => 12:  23.3282 (0.80)
 0 => 13:  28.2310 (0.97)
 0 => 14:   8.9147 (0.31)
 0 => 15:   8.0444 (0.28)
 0 => 16:  10.1345 (0.35)
 0 => 17:  13.7918 (0.47)
 0 => 18:  18.0561 (0.62)
 0 => 19:  22.5567 (0.77)
 0 => 20:  27.1780 (0.93)
 0 => 21:  13.5391 (0.46)
 0 => 22:  12.1910 (0.42)
 0 => 23:  12.9131 (0.44)
 0 => 24:  15.3619 (0.53)
 0 => 25:  18.7894 (0.64)
 0 => 26:  22.7567 (0.78)
 0 => 27:  27.0036 (0.93)
 0 => 28:  18.2439 (0.63)
 0 => 29:  16.6445 (0.57)
 0 => 30:  16.5799 (0.57)
 0 => 31:  18.0691 (0.62)
 0 => 32:  20.6243 (0.71)
 0 => 33:  23.8888 (0.82)
 0 => 34:  27.6188 (0.95)
 1 =>  2:   4.9995 (0.17)
 1 =>  3:   9.9931 (0.34)
 1 =>  4:  14.9642 (0.51)
 1 =>  5:  19.9459 (0.68)
 1 =>  6:  2

# Diskussion
Der hier gezeigte Algorithmus ist stark von den gewählten Punkten und der Kamerakalibrierung abhängig.
Da auch im _OpenCV_ Ansatz einige Unstimmigkeiten zu finden sind, lässt dies auf ein Problem bei den Eingangsparametern schließen.

Eine mögliche Erklärung könnten einige ungenaue Angabe der korrespondierenden Punkte dienen. Jedoch brachte die Verwendung von SIFT in ersten Versuchen keine merkliche Besserung.

Eine weitere und sehr wahrscheinliche Ursache könnte die Kalibrierung der Kamera darstellen. Da beim Kalibrieren, das Blatt nicht allzu stark bewegt wurde und vor allem wenig seitlich gekippt, können die Projektionsmatrizen den Grund für die großen Unterschiede darstellen.
Desweiteren war das verwendete Blatt nicht besonders stabil und bildete somit manchmal eine leichte Krümmung.
Es wird daher empfohlen die Kalibrierung erneut durchzuführen.

Desweiteren kann eine Verzerrung durch die Kamera einen großen Einfluss auf die Triangulierung haben, weshalb ein Entzerren während der Kalibrierung ebenfalls eine Lösung darstellen könnte.

Zusätzlich war das Muster des verwendeten Schachbrett quadratisch, was eventuell auch einen Einfluss auf die Kalibrierung der Kamera haben könnte. Dazu konnte aber aktuell noch nichts gefunden werden.

# Quellen
<a name="paper"></a> [1] Hartley/Zisserman, Multiple View Geometry, p.310-318

[2] https://www.youtube.com/watch?v=UZlRhEUWSas

[3] https://en.wikipedia.org/wiki/Triangulation_(computer_vision)

[4] https://gist.github.com/cr333/0d0e6c256f93c0ed7bd2

[5] https://uni-tuebingen.de/fakultaeten/mathematisch-naturwissenschaftliche-fakultaet/fachbereiche/informatik/lehrstuehle/autonomous-vision/lectures/computer-vision/

[6] https://docs.opencv.org/3.4/da/de9/tutorial_py_epipolar_geometry.html

[7] https://www.changjiangcai.com/files/text-books/Richard_Hartley_Andrew_Zisserman-Multiple_View_Geometry_in_Computer_Vision-EN.pdf

[8] https://medium.com/@insight-in-plain-sight/estimating-the-homography-matrix-with-the-direct-linear-transform-dlt-ec6bbb82ee2b

[9] https://glowingpython.blogspot.com/2011/06/svd-decomposition-with-numpy.html